In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 

w_time = 5
firstname = input("enter first name")
lastname = input("enter last name")
name = f"{firstname} {lastname}"
name = name.lower()


In [2]:
def login(driver):
    driver.get("https://www.linkedin.com/login")
    username = ""           #enter your username for linkedin login
    password = ""                           #enter your password for linkedin account

    username_input = driver.find_element(By.ID, "username")
    username_input.send_keys(username)
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(password)

    sign_in_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    sign_in_button.click()

    driver.implicitly_wait(w_time)  # may need to increase wait time here if login id used too many times

def get_data(driver):

    search_bar = driver.find_element(By.XPATH, "//input[@placeholder='Search']")  
    search_bar.send_keys(f"{firstname} {lastname}")
    search_bar.send_keys(Keys.ENTER)
    driver.implicitly_wait(w_time)
    people_button = driver.find_element(By.XPATH, "//button[text()='People']") 
    people_button.click()

    driver.implicitly_wait(w_time) 

    extracted_data = []
    profile_cards = driver.find_elements(By.CLASS_NAME, "reusable-search__result-container")
    count = 0

    for card in profile_cards:
        link = card.find_element(By.CLASS_NAME, "app-aware-link").get_attribute("href") 
        data = card.text.split("\n")
        if(len(data)==7):
            i = 0
        else:
            i = 1
        if(data[0+i].lower() == name):
            count+=1
            details = {"Name":data[0+i],"Headline":data[4+i],"Location":data[5+i],"Linkedin":link}
            extracted_data.append(details)
    return extracted_data,count

def scrape_data():
    driver=webdriver.Chrome()
    try:
        login(driver)
        data,cnt = get_data(driver)
        with open('extracted_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Name', 'Headline', 'Location','Linkedin']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for profile_data in data:
                writer.writerow(profile_data)
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    time.sleep(w_time)
    driver.quit()
    print("data extracted successfully, found "+str(cnt)+" matching results")
    return 


In [3]:
scrape_data()

data extracted successfully, found 8 matching results
